In [2]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pylab import rcParams
from scipy.ndimage.interpolation import rotate, zoom, shift
import time
from tqdm import tqdm_notebook

In [21]:
imgPaths = glob.glob("../imgs/*/w1_*")
labelled = glob.glob("../imgs/*/l_*")
print("Number of labelled images = %d." % len(labelled))

Number of labelled images = 346.


In [16]:
pathAug = "augmented"
if not os.path.exists(pathAug):
    os.makedirs(pathAug)
labels = glob.glob("../imgs/whale_*/lS_*.jpg")
nObs = len(labels)
print("Have %d labels"%nObs)

Have 346 labels


In [8]:
egImg = Image.open(imgPaths[0])
ar = (egImg.size[0]/float(egImg.size[1]))
newWidth = 450
newHeight = int(newWidth/ar)
print("reshaping to %d by %d" %(newWidth,newHeight))

rcParams['figure.figsize'] = 9, ar*9

reshaping to 450 by 300


In [30]:
def resizeImages(labeledOrOriginal):

    s = str(labeledOrOriginal)
    imgPaths = glob.glob("../imgs/*/"+s+"_*")
    count = 0
    nObs = len(imgPaths)
    
    for i in tqdm_notebook(range(nObs)):
        imgPath = imgPaths[i]
        dst = imgPath.replace(s+"_",s+"S_")
        if os.path.exists(dst) == False:
            img = Image.open(imgPath)
            img = img.resize((newWidth,newHeight),Image.ANTIALIAS)
            img.save(dst)TrueTrueTrue
            count += 1True
    print ("Added %d more resized images " % count)
#resizeImages("l") # downscale labelTrues
#resizeImages("w1") # downscale originals


Added 11468 more resized images 


In [18]:
averageTimes = []
imgNo = 0
nImgs = len(labels)
nAug = 10
for i in tqdm_notebook(range(nImgs)[:],"Images to augment"):
    y = labels[i]
    x = y.replace("lS_","w1S_")
    imgX, imgY = Image.open(x), Image.open(y)
    r = np.array(imgY)
    notRedYellow = (r[:,:,0] < 200) & (r[:,:,1] < 200) | (r[:,:,2] > 100)
    r[notRedYellow] = 0
    redYellow = np.invert(notRedYellow)
    #r[redYellow] = 255
    r = Image.fromarray(r)
    
    for i in tqdm_notebook(range(nAug),desc=""):
        
        start = time.time()
        angle = np.random.uniform(-30,30)
        zoomFactor = np.random.uniform(0.9,1.1)
        imgX_, r_ = imgX.copy(), r.copy()
        shiftX, shiftY, _ = np.random.normal(0,25,3)
        
        
        if np.random.uniform() < 0.5:
            imgX_, r_ = [img.transpose(Image.FLIP_LEFT_RIGHT) for img in [imgX_,r_]]
        imgX_ = rotate(imgX_,angle,mode="nearest")
        r_ = rotate(r_,angle,mode="constant")
        
        imgX_ = shift(imgX_,(shiftX, shiftY,0),mode="constant")
        r_ = shift(r_,(shiftX, shiftY,0),mode="constant")
        
        imgX_, r_ = [Image.fromarray(zoom(img,zoomFactor)) for img in [imgX_,r_]]
        imgX_, r_ = [img.resize((newWidth,newHeight)) for img in [imgX_,r_]]
        
        imgX_.save("augmented/x_"+str(imgNo)+".png")
        r_.save("augmented/y_"+str(imgNo)+".png")
        
        elapsed = time.time() - start
        averageTimes.append(elapsed)
        imgNo += 1


/usr/local/lib/python3.4/dist-packages/scipy/ndimage/interpolation.py:549: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
